# Обработка текста на примере предсказание тематики новостей

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import re
import string
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
paths = {
    'MODELS':'models',
    'LR':os.path.join('models','LogisticRegression'),
    'FASTTEXT':os.path.join('models','fasttext'),
    'DATA':'data',
    'TRAIN':os.path.join('data','train'),
    'TEST':os.path.join('data','test')
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'nt':  
            print(path)
            !mkdir {path}
        elif os.name == 'posix':
            !mkdir {path}
        

### Загрузка данных

In [ ]:
df = pd.DataFrame(columns=['title','text','topic','tags'])
for df_text in pd.read_csv(os.path.join(paths['DATA'],'lenta-ru-news.csv'), chunksize=1000):
    df_text = df_text[['title','text','topic','tags']]
    df = pd.concat([df,df_text])
df.head(5)

In [ ]:
df.loc[df.tags=='Украина','topic'] = 'Украина'

In [ ]:
df_text = df[["text",'topic']].dropna()

In [ ]:
query = 'topic!="Крым"&topic!="Сочи"&topic!="МедНовости"&topic!="ЧМ-2014"&topic!="Оружие"&topic!="Библиотека" \
                &topic!="Легпром"&topic!="Культпросвет "&topic!="69-я параллель"'
df_text = df_text.query(query)
df_text.topic = df_text.topic.apply(lambda x: x.replace(' ','_'))
df_text.head(5)

### Обработка данных

Обрабатывать данные будем двумя способами.
1. с помощью метода simple_preproces библиотеки gensim 
2. с помощью написанного класса CleanText, используя simple преобразования
3. с помощью написанного класса CleanText, применяя удаления стопслов и стеминга

In [ ]:
from gensim.utils import simple_preprocess

In [ ]:
from collections import defaultdict
class CleanText(str):
    def __init__(self,text=''):
        self.options = defaultdict(bool)
    def __call__(self,text=''):
        if self.options['full_transform']:
            text = self.remove_whit_re(text)
            text = self.remove_punctuation(text)
            text = self.remove_stopwords(text)
            text = self.apply_stemming(text)        
            return text
        text = self.remove_whit_re(text)
        text = self.remove_punctuation(text)
        text = self.remove_one_char(text)
        return text              
    
    
    def set_options(self,
                    simple=False, 
                    full_transform=False, 
                    ):
        self.options['simple']=simple
        self.options['full_transform']=full_transform         
    
    
    def remove_one_char(self, text:str=''):
        if self:
            text = self+' '+text
        return ' '.join([word for word in text.split(' ') if len(word)>1 or len(word)>18])
    
    
    def remove_inicial(self,text:str=''):
        if self:
            text = self+' '+text
        return re.sub("\.*\s[A-ZА-Я]\.","",text)
        
    
    
    def remove_whit_re(self,text:str=''):
        if self:
            text = self+' '+text
        return re.sub("[A-ZА-Я]\.|http[://A-Za-zА-Яа-я\.]*|www.*[A-Za-z]+\.[a-zA-Z]+","",text)
    
    
    def lower(self,text:str):
        if self:
            text = self+' '+text
        return text.lower()
    
   
    def remove_punctuation(self,text:str=''):
        if self:
            text = self+' '+text
        whitespace = string.whitespace.replace(' ','')
        punctuations = string.punctuation+'«»'
        digits = string.digits
        schars=whitespace+punctuations+digits  
        text = text.replace('-',' ')
        return "".join([char for char in text.lower() \
                        if char not in schars])
    
    
    def remove_stopwords(self,text:str=''):
        if self:
            text = self+' '+text
        stemmer = SnowballStemmer('russian')
        stop_words = stopwords.words('russian')
        return " ".join([stemmer.stem(word) for word in text.split(' ') if word not in stop_words])
   

  
    def apply_stemming(self,text:str=''):
        if self:
            text = self+' '+text
        stemmer = SnowballStemmer('russian')
        return " ".join(stemmer.stem(word) for word in text.split(' '))
    
    
    def transform(self,lst_good_words:list,text:str=''):
        if self:
            text = self+' '+text
        return " ".join([words for words in text.split(' ') if words in lst_good_words])

In [ ]:
row = []
for text in tqdm(df_text.text.values):
    row.append(' '.join(simple_preprocess(text)))

In [ ]:
row_1 = []
cleaner = CleanText()
for text in tqdm(df_text.text.values):
    row_1.append(cleaner(text))

In [ ]:
df_gensim = pd.DataFrame(list(zip(row,df_text.topic.values)), columns=['text','topic']).dropna()
df_cleaner = pd.DataFrame(list(zip(row_1,df_text.topic.values)), columns=['text','topic']).dropna()

In [ ]:
df_cleaner.head(5)

### Сохранение данных

In [ ]:
df_gensim.to_csv(os.path.join(paths['DATA'],'gensim.csv'))

In [ ]:
df_cleaner.to_csv(os.path.join(paths['DATA'],'cleaner.csv'))

В итоге написанный класс в среднем на 20 процентов быстрее предобробатывает русский текст, и также имеет возможноть удалять url адреса